# ISY5002 PSUPR/PRMLS CA1
## Data set is sourced from https://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results
## Data set label for column 5 onward are from https://github.com/kronusme/dota2-api/blob/master/data/heroes.json

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC

In [2]:
raw_data_train = pd.read_csv('..\dataset\dota2Train.csv')
raw_data_test = pd.read_csv('..\dataset\dota2Test.csv')

In [4]:
raw_data_train.head()

,Won,Cluster ID,Game Mode,Game Type,Anti-Mage,Axe,Bane,Bloodseeker,Crystal Maiden,Drow Ranger,...,Legion Commander,Techies,Ember Spirit,Earth Spirit,Abyssal Underlord,Terrorblade,Phoenix,Oracle,Winter Wyvern,Arc Warden
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train = raw_data_train.drop(["Won", "Cluster ID", "Game Mode", "Game Type"], axis=1)
y_train = raw_data_train["Won"]
x_test = raw_data_test.drop(["Won", "Cluster ID", "Game Mode", "Game Type"], axis=1)
y_test = raw_data_test["Won"]

In [6]:
x_train.head()

,Anti-Mage,Axe,Bane,Bloodseeker,Crystal Maiden,Drow Ranger,Earthshaker,Juggernaut,Mirana,Shadow Fiend,...,Legion Commander,Techies,Ember Spirit,Earth Spirit,Abyssal Underlord,Terrorblade,Phoenix,Oracle,Winter Wyvern,Arc Warden
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,-1,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,-1,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_train.head()

0   -1
1    1
2    1
3    1
4   -1
Name: Won, dtype: int64

In [8]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92650 entries, 0 to 92649
Columns: 113 entries, Anti-Mage to Arc Warden
dtypes: int64(113)
memory usage: 79.9 MB


In [9]:
linsvc = LinearSVC(random_state=42, tol=1e-5, max_iter=5000, verbose=1)
linsvc.fit(x_train, y_train)

[LibLinear]

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=42, tol=1e-05,
          verbose=1)

In [10]:
print(linsvc.predict(x_test))

[-1  1  1 ...  1  1  1]


In [11]:
print(linsvc.score(x_test, y_test))

0.5973382552943463
